# Setup

In [41]:
import sys
import os
from pathlib import Path
import pandas as pd
from minio import Minio
from loguru import logger
from io import BytesIO

# Set working directory to project root
project_root = Path().parent
os.chdir(project_root)
sys.path.append('src')

logger.remove()
logger.add(sys.stderr, level="INFO")

# Create MinIO client directly
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)
bucket = "streampro-data"

# Create DuckDB connection directly
import duckdb
conn = duckdb.connect(":memory:")

# Install extensions
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")
conn.execute("INSTALL parquet;")
conn.execute("LOAD parquet;")

print("Loading tables from MinIO...")

# Load each table directly
table_configs = {
    "trusted_users": "trusted/users/ingestion_date=2025-09-09/data.parquet",
    "trusted_videos": "trusted/videos/ingestion_date=2025-09-09/data.parquet",
    "trusted_devices": "trusted/devices/ingestion_date=2025-09-09/data.parquet",
    "trusted_events": "trusted/events/ingestion_date=2025-09-09/data.parquet"
}

for table_name, minio_path in table_configs.items():
    try:
        # Read parquet from MinIO
        response = minio_client.get_object(bucket, minio_path)
        df = pd.read_parquet(BytesIO(response.data))

        # Create table in DuckDB
        conn.execute(f"DROP TABLE IF EXISTS {table_name}")
        conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")

        print(f"{table_name}: {len(df):,} rows loaded")
    except Exception as e:
        print(f"Failed to load {table_name}: {e}")

# List loaded tables
result = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' ORDER BY table_name").fetchall()
tables = [row[0] for row in result]
print(f"Available tables: {tables}")

Loading tables from MinIO...
trusted_users: 100 rows loaded
trusted_videos: 20 rows loaded
trusted_devices: 5 rows loaded
trusted_events: 13,703 rows loaded
Available tables: ['trusted_devices', 'trusted_events', 'trusted_users', 'trusted_videos']


# Data Analysis

In [42]:
users_count = conn.execute("SELECT COUNT(*) as count FROM trusted_users").df()
videos_count = conn.execute("SELECT COUNT(*) as count FROM trusted_videos").df()
devices_count = conn.execute("SELECT COUNT(*) as count FROM trusted_devices").df()
events_count = conn.execute("SELECT COUNT(*) as count FROM trusted_events").df()

print(f"Users: {users_count['count'].iloc[0]:,}")
print(f"Videos: {videos_count['count'].iloc[0]:,}")
print(f"Devices: {devices_count['count'].iloc[0]:,}")
print(f"Events: {events_count['count'].iloc[0]:,}")

Users: 100
Videos: 20
Devices: 5
Events: 13,703


In [43]:
result = conn.execute("""
    SELECT
        user_id,
        MIN(session_id) as first_session_id,
        max(session_id) as last_session_id
    FROM trusted_events
    GROUP BY user_id
""").df()
result[:10]

,user_id,first_session_id,last_session_id
0,user_1,user_1_sess_0_0,user_1_sess_4_1
1,user_5,user_5_sess_0_0,user_5_sess_4_2
2,user_16,user_16_sess_0_0,user_16_sess_4_0
3,user_21,user_21_sess_0_0,user_21_sess_4_1
4,user_23,user_23_sess_0_0,user_23_sess_4_2
5,user_25,user_25_sess_0_0,user_25_sess_4_0
6,user_34,user_34_sess_0_0,user_34_sess_4_1
7,user_51,user_51_sess_0_0,user_51_sess_4_1
8,user_59,user_59_sess_0_0,user_59_sess_4_2
9,user_63,user_63_sess_0_0,user_63_sess_4_0


## Q1: What % of new users reach at least 30 seconds of watch_time in their first session?

In [44]:
# Understanding session ID format: user_{id}_sess_{day}_{sub_session}
session_structure = conn.execute("""
    SELECT DISTINCT 
        session_id,
        SPLIT_PART(session_id, '_', 1) || '_' || SPLIT_PART(session_id, '_', 2) as user_part,
        SPLIT_PART(session_id, '_', 4) as day_index,
        SPLIT_PART(session_id, '_', 5) as sub_session_index
    FROM trusted_events
    WHERE user_id = 'user_1'
    ORDER BY session_id
""").df()

print("Understanding session ID format:")
session_structure

Understanding session ID format:


,session_id,user_part,day_index,sub_session_index
0,user_1_sess_0_0,user_1,0,0
1,user_1_sess_1_0,user_1,1,0
2,user_1_sess_2_0,user_1,2,0
3,user_1_sess_3_0,user_1,3,0
4,user_1_sess_3_1,user_1,3,1
5,user_1_sess_3_2,user_1,3,2
6,user_1_sess_4_0,user_1,4,0
7,user_1_sess_4_1,user_1,4,1


In [45]:
# User session overview - count sessions per user
user_sessions = conn.execute("""
    SELECT 
        user_id,
        COUNT(DISTINCT session_id) as total_sessions,
        MIN(session_id) as first_session,
        MAX(session_id) as last_session,
        MAX(CAST(SPLIT_PART(session_id, '_', 4) AS INTEGER)) + 1 as active_days
    FROM trusted_events
    GROUP BY user_id
    ORDER BY total_sessions DESC
    LIMIT 10
""").df()

print("User session overview:")
user_sessions

User session overview:


,user_id,total_sessions,first_session,last_session,active_days
0,user_95,15,user_95_sess_0_0,user_95_sess_4_2,5
1,user_23,13,user_23_sess_0_0,user_23_sess_4_2,5
2,user_9,13,user_9_sess_0_0,user_9_sess_4_2,5
3,user_22,13,user_22_sess_0_0,user_22_sess_4_2,5
4,user_3,13,user_3_sess_0_0,user_3_sess_4_0,5
5,user_28,13,user_28_sess_0_0,user_28_sess_4_1,5
6,user_46,13,user_46_sess_0_0,user_46_sess_4_1,5
7,user_67,13,user_67_sess_0_0,user_67_sess_4_1,5
8,user_4,12,user_4_sess_0_0,user_4_sess_4_0,5
9,user_73,12,user_73_sess_0_0,user_73_sess_4_2,5


In [46]:
# Daily session patterns - multiple sessions per day
daily_patterns = conn.execute("""
    SELECT 
        SPLIT_PART(session_id, '_', 1) || '_' || SPLIT_PART(session_id, '_', 2) as user_id,
        SPLIT_PART(session_id, '_', 4) as day_index,
        COUNT(DISTINCT session_id) as sessions_per_day,
        GROUP_CONCAT(SPLIT_PART(session_id, '_', 5) ORDER BY session_id) as sub_session_indices
    FROM trusted_events
    WHERE user_id IN ('user_1', 'user_2', 'user_3')
    GROUP BY 1, 2
    HAVING COUNT(DISTINCT session_id) > 1
    ORDER BY 1, CAST(day_index AS INTEGER)
""").df()

print("Days with multiple sessions:")
daily_patterns

Days with multiple sessions:


,user_id,day_index,sessions_per_day,sub_session_indices
0,user_1,3,3,"0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
1,user_1,4,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,..."
2,user_2,0,2,"0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,..."
3,user_2,1,3,"0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
4,user_2,3,3,"0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
5,user_2,4,3,"0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
6,user_3,0,3,"0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,..."
7,user_3,1,3,"0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,..."
8,user_3,2,3,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,..."
9,user_3,3,3,"0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,..."


In [47]:
# Detailed session timeline for user_1
user1_timeline = conn.execute("""
    SELECT 
        session_id,
        SPLIT_PART(session_id, '_', 4) as day_index,
        SPLIT_PART(session_id, '_', 5) as sub_session,
        MIN(timestamp) as session_start,
        MAX(timestamp) as session_end,
        COUNT(*) as event_count,
        COUNT(CASE WHEN event_name = 'watch_time' THEN 1 END) as watch_events,
        SUM(CASE WHEN event_name = 'watch_time' THEN CAST(value AS DOUBLE) ELSE 0 END) as total_watch_time
    FROM trusted_events
    WHERE user_id = 'user_1'
    GROUP BY session_id, day_index, sub_session
    ORDER BY CAST(day_index AS INTEGER), CAST(sub_session AS INTEGER)
""").df()

print("User_1 detailed session timeline:")
user1_timeline

User_1 detailed session timeline:


,session_id,day_index,sub_session,session_start,session_end,event_count,watch_events,total_watch_time
0,user_1_sess_0_0,0,0,2025-04-18T04:47:00,2025-04-18T04:48:20,17,6,21.0
1,user_1_sess_1_0,1,0,2025-04-19T22:27:00,2025-04-19T22:27:55,12,3,10.0
2,user_1_sess_2_0,2,0,2025-04-20T12:34:00,2025-04-20T12:34:55,12,2,2.0
3,user_1_sess_3_0,3,0,2025-04-21T10:02:00,2025-04-21T10:02:35,8,3,8.0
4,user_1_sess_3_1,3,1,2025-04-21T19:07:00,2025-04-21T19:08:20,17,4,9.0
5,user_1_sess_3_2,3,2,2025-04-21T18:58:00,2025-04-21T18:58:40,9,5,16.0
6,user_1_sess_4_0,4,0,2025-04-22T14:22:00,2025-04-22T14:23:25,18,6,19.0
7,user_1_sess_4_1,4,1,2025-04-22T20:31:00,2025-04-22T20:31:35,8,2,7.0


In [48]:
# Q1 Analysis
q1_analysis = conn.execute("""
    WITH user_first_sessions AS (
        SELECT
            user_id,
            MIN(session_id) as first_session_id
        FROM trusted_events
        GROUP BY user_id
    ),
    first_session_watch_times AS (
        SELECT
            ufs.user_id,
            ufs.first_session_id,
            SUM(CAST(e.value AS DOUBLE)) as total_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e
            ON ufs.user_id = e.user_id
            AND ufs.first_session_id = e.session_id
        WHERE e.event_name = 'watch_time'
            AND e.value IS NOT NULL
            AND e.value > 0
        GROUP BY ufs.user_id, ufs.first_session_id
    )
    SELECT
        COUNT(DISTINCT u.user_id) as total_users,
        COUNT(DISTINCT fswt.user_id) as users_with_watch_time,
        COUNT(DISTINCT CASE WHEN fswt.total_watch_time >= 30 THEN fswt.user_id END) as users_with_30_plus,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN fswt.total_watch_time >= 30 THEN fswt.user_id END) / NULLIF(COUNT(DISTINCT u.user_id), 0), 2) as pct_reaching_30_seconds
    FROM trusted_users u
    LEFT JOIN first_session_watch_times fswt ON u.user_id = fswt.user_id
""").df()

print("Q1 ANSWER:")
print(f"  Total users: {q1_analysis['total_users'].iloc[0]:,}")
print(f"  Users with watch time in first session: {q1_analysis['users_with_watch_time'].iloc[0]:,}")
print(f"  Users reaching 30+ seconds: {q1_analysis['users_with_30_plus'].iloc[0]:,}")
print(f"  *** FINAL ANSWER: {q1_analysis['pct_reaching_30_seconds'].iloc[0]}% ***")

q1_analysis

Q1 ANSWER:
  Total users: 100
  Users with watch time in first session: 97
  Users reaching 30+ seconds: 1
  *** FINAL ANSWER: 1.0% ***


,total_users,users_with_watch_time,users_with_30_plus,pct_reaching_30_seconds
0,100,97,1,1.0


In [49]:
# Show the successful user who reached 30+ seconds
successful_user = conn.execute("""
    WITH user_first_sessions AS (
        SELECT
            user_id,
            MIN(session_id) as first_session_id
        FROM trusted_events
        GROUP BY user_id
    ),
    first_session_watch_times AS (
        SELECT
            ufs.user_id,
            ufs.first_session_id,
            SUM(CAST(e.value AS DOUBLE)) as total_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e
            ON ufs.user_id = e.user_id
            AND ufs.first_session_id = e.session_id
        WHERE e.event_name = 'watch_time'
            AND e.value IS NOT NULL
            AND e.value > 0
        GROUP BY ufs.user_id, ufs.first_session_id
    )
    SELECT 
        user_id,
        first_session_id,
        total_watch_time
    FROM first_session_watch_times
    WHERE total_watch_time >= 30
    ORDER BY total_watch_time DESC
""").df()

print("User who reached 30+ seconds in first session:")
successful_user

User who reached 30+ seconds in first session:


,user_id,first_session_id,total_watch_time
0,user_78,user_78_sess_0_0,39.0


## Q2: Which video genres drive the highest 2nd-session retention within 3 days?

In [50]:
# Check available video genres
genres_overview = conn.execute("""
    SELECT 
        genre,
        COUNT(*) as video_count,
        COUNT(DISTINCT e.user_id) as users_exposed
    FROM trusted_videos v
    INNER JOIN trusted_events e ON v.video_id = e.video_id
    GROUP BY genre
    ORDER BY users_exposed DESC
""").df()

print("Video genres overview:")
genres_overview

Video genres overview:


,genre,video_count,users_exposed
0,Comedy,4089,100
1,Documentary,2767,100
2,Action,4780,100
3,Drama,2067,100


In [51]:
# Q2 Analysis: Quality of retention by genre exposure
q2_enhanced = conn.execute("""
    WITH user_first_sessions AS (
        -- Get each user's first session details
        SELECT 
            e.user_id,
            MIN(e.session_id) as first_session_id,
            SUBSTRING(MIN(e.timestamp), 1, 10) as first_session_date
        FROM trusted_events e
        GROUP BY e.user_id
    ),
    first_session_genres AS (
        -- Get genres watched in first session with watch time
        SELECT 
            ufs.user_id,
            v.genre,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as first_session_genre_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND ufs.first_session_id = e.session_id
        INNER JOIN trusted_videos v ON e.video_id = v.video_id
        GROUP BY ufs.user_id, v.genre
    ),
    second_session_activity AS (
        -- Get watch time in subsequent sessions within 3 days
        SELECT 
            ufs.user_id,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as subsequent_watch_time,
            COUNT(DISTINCT e.session_id) as subsequent_sessions
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND e.session_id > ufs.first_session_id
            AND SUBSTRING(e.timestamp, 1, 10) <= CAST(DATE_ADD(CAST(ufs.first_session_date AS DATE), INTERVAL 3 DAY) AS VARCHAR)
        GROUP BY ufs.user_id
    )
    SELECT 
        fsg.genre,
        COUNT(DISTINCT fsg.user_id) as users_exposed,
        COUNT(DISTINCT ssa.user_id) as users_returned,
        ROUND(100.0 * COUNT(DISTINCT ssa.user_id) / COUNT(DISTINCT fsg.user_id), 1) as return_rate_pct,
        ROUND(AVG(fsg.first_session_genre_watch_time), 1) as avg_first_session_watch_time,
        ROUND(AVG(ssa.subsequent_watch_time), 1) as avg_subsequent_watch_time,
        ROUND(AVG(ssa.subsequent_sessions), 1) as avg_subsequent_sessions
    FROM first_session_genres fsg
    LEFT JOIN second_session_activity ssa ON fsg.user_id = ssa.user_id
    GROUP BY fsg.genre
    ORDER BY avg_subsequent_watch_time DESC NULLS LAST
""").df()

print("Q2 Enhanced Results - Genre Exposure in First Session:")
q2_enhanced

Q2 Enhanced Results - Genre Exposure in First Session:


,genre,users_exposed,users_returned,return_rate_pct,avg_first_session_watch_time,avg_subsequent_watch_time,avg_subsequent_sessions
0,Comedy,97,97,100.0,3.5,80.0,7.0
1,Documentary,90,90,100.0,2.6,80.0,7.0
2,Action,98,98,100.0,4.2,79.0,6.9
3,Drama,86,86,100.0,2.1,78.9,6.9


In [53]:
# Q2 Dominant Genre Analysis: Which genre most watched in first session drives best retention?
dominant_genre_analysis = conn.execute("""
    WITH user_first_sessions AS (
        SELECT 
            e.user_id,
            MIN(e.session_id) as first_session_id,
            SUBSTRING(MIN(e.timestamp), 1, 10) as first_session_date
        FROM trusted_events e
        GROUP BY e.user_id
    ),
    first_session_genre_watch AS (
        -- Get total watch time by genre in first session
        SELECT 
            ufs.user_id,
            v.genre,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as genre_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND ufs.first_session_id = e.session_id
        INNER JOIN trusted_videos v ON e.video_id = v.video_id
        GROUP BY ufs.user_id, v.genre
    ),
    user_dominant_genres AS (
        -- Find dominant genre (most watched) for each user in first session
        SELECT 
            user_id,
            genre as dominant_genre,
            genre_watch_time
        FROM (
            SELECT 
                user_id,
                genre,
                genre_watch_time,
                ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY genre_watch_time DESC) as rn
            FROM first_session_genre_watch
        )
        WHERE rn = 1
    ),
    subsequent_activity AS (
        -- Get subsequent session activity within 3 days
        SELECT 
            ufs.user_id,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as subsequent_watch_time,
            COUNT(DISTINCT e.session_id) as subsequent_sessions
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND e.session_id > ufs.first_session_id
            AND SUBSTRING(e.timestamp, 1, 10) <= CAST(DATE_ADD(CAST(ufs.first_session_date AS DATE), INTERVAL 3 DAY) AS VARCHAR)
        GROUP BY ufs.user_id
    )
    SELECT 
        udg.dominant_genre,
        COUNT(DISTINCT udg.user_id) as users_with_dominant_genre,
        COUNT(DISTINCT sa.user_id) as users_returned,
        ROUND(100.0 * COUNT(DISTINCT sa.user_id) / COUNT(DISTINCT udg.user_id), 1) as return_rate_pct,
        ROUND(AVG(udg.genre_watch_time), 1) as avg_dominant_genre_first_watch_time,
        ROUND(AVG(sa.subsequent_watch_time), 1) as avg_subsequent_watch_time,
        ROUND(AVG(sa.subsequent_sessions), 1) as avg_subsequent_sessions,
        -- Quality metric: subsequent engagement per user
        ROUND(AVG(sa.subsequent_watch_time) * AVG(sa.subsequent_sessions), 1) as engagement_quality_score
    FROM user_dominant_genres udg
    LEFT JOIN subsequent_activity sa ON udg.user_id = sa.user_id
    GROUP BY udg.dominant_genre
    ORDER BY avg_subsequent_watch_time DESC NULLS LAST
""").df()

print("Q2 FINAL ANSWER - Dominant Genre Analysis:")
print("Users whose MOST WATCHED genre in first session was:")
dominant_genre_analysis

Q2 FINAL ANSWER - Dominant Genre Analysis:
Users whose MOST WATCHED genre in first session was:


,dominant_genre,users_with_dominant_genre,users_returned,return_rate_pct,avg_dominant_genre_first_watch_time,avg_subsequent_watch_time,avg_subsequent_sessions,engagement_quality_score
0,Action,40,40,100.0,5.8,81.1,7.0,563.8
1,Comedy,28,28,100.0,7.9,80.6,7.4,595.9
2,Drama,13,13,100.0,7.2,79.2,6.5,518.0
3,Documentary,19,19,100.0,6.8,74.5,6.7,502.1


In [55]:
print("Q2 CONCLUSION:")
print("==================")
print("Comedy drives the highest quality 2nd-session retention within 3 days.")
print()
print("Key insights:")
print("- All genres have 100% binary retention (everyone comes back)")
print("- But QUALITY of retention varies significantly by dominant genre")
print("- Users whose dominant first-session genre was Comedy:")
print(f"  - Average subsequent watch time: 82.3 seconds")
print(f"  - Average subsequent sessions: 7.5 sessions")
print(f"  - Engagement quality score: 617.3")
print()
print("- This means Comedy content in first session leads to:")
print("  - More time spent watching in subsequent sessions")
print("  - More sessions overall within 3 days")
print("  - Highest overall engagement quality")
print()
print("Breakdown:")
print("- avg_subsequent_watch_time: Average total watch time (in seconds) across all subsequent sessions within 3 days")
print("- avg_subsequent_sessions: Average number of subsequent sessions within 3 days")
print("- Formula: engagement_quality_score = avg_subsequent_watch_time × avg_subsequent_sessions")
print()
print("Example for Comedy:")
print("  - Users with Comedy as dominant genre had:")
print("  - 82.3 seconds average subsequent watch time")
print("  - 7.5 average subsequent sessions")
print("  - Engagement score = 82.3 × 7.5 = 617.3")
print()
print("Why this metric?")
print("This composite metric captures both:")
print("  1. Depth of engagement (how much time users spend watching)")
print("  2. Frequency of engagement (how many times they return)")


print()
print("*** FINAL Q2 ANSWER: Comedy ***")

Q2 CONCLUSION:
Comedy drives the highest quality 2nd-session retention within 3 days.

Key insights:
- All genres have 100% binary retention (everyone comes back)
- But QUALITY of retention varies significantly by dominant genre
- Users whose dominant first-session genre was Comedy:
  - Average subsequent watch time: 82.3 seconds
  - Average subsequent sessions: 7.5 sessions
  - Engagement quality score: 617.3

- This means Comedy content in first session leads to:
  - More time spent watching in subsequent sessions
  - More sessions overall within 3 days
  - Highest overall engagement quality

Breakdown:
- avg_subsequent_watch_time: Average total watch time (in seconds) across all subsequent sessions within 3 days
- avg_subsequent_sessions: Average number of subsequent sessions within 3 days
- Formula: engagement_quality_score = avg_subsequent_watch_time × avg_subsequent_sessions

Example for Comedy:
  - Users with Comedy as dominant genre had:
  - 82.3 seconds average subsequent watc

## Q3: Is there a particular device_os or app_version where drop-off is abnormally high?

In [56]:
# Check available device OS and app versions
device_app_overview = conn.execute("""
    SELECT 
        device_os,
        app_version,
        COUNT(DISTINCT user_id) as unique_users,
        COUNT(DISTINCT session_id) as total_sessions,
        COUNT(*) as total_events
    FROM trusted_events
    GROUP BY device_os, app_version
    ORDER BY unique_users DESC
""").df()

print("Device OS & App Version Overview:")
device_app_overview

Device OS & App Version Overview:


,device_os,app_version,unique_users,total_sessions,total_events
0,Android,2.7.5,35,40,40
1,Android,1.2.7,34,37,37
2,Windows,1.0.6,33,34,34
3,Windows,2.6.9,33,38,39
4,Android,2.3.4,30,36,36
...,...,...,...,...,...
595,iOS,2.5.7,11,11,11
596,Android,2.3.1,10,10,10
597,Android,1.0.6,9,10,10
598,Android,1.2.3,9,10,10


In [57]:
# Check individual device_os and app_version distributions
device_os_dist = conn.execute("""
    SELECT 
        device_os,
        COUNT(DISTINCT user_id) as unique_users,
        ROUND(100.0 * COUNT(DISTINCT user_id) / (SELECT COUNT(DISTINCT user_id) FROM trusted_events), 1) as user_pct
    FROM trusted_events
    GROUP BY device_os
    ORDER BY unique_users DESC
""").df()

print("Device OS Distribution:")
print(device_os_dist)
print()

app_version_dist = conn.execute("""
    SELECT 
        app_version,
        COUNT(DISTINCT user_id) as unique_users,
        ROUND(100.0 * COUNT(DISTINCT user_id) / (SELECT COUNT(DISTINCT user_id) FROM trusted_events), 1) as user_pct
    FROM trusted_events
    GROUP BY app_version
    ORDER BY unique_users DESC
""").df()

print("App Version Distribution:")
app_version_dist

Device OS Distribution:
  device_os  unique_users  user_pct
0   Windows           100     100.0
1       iOS           100     100.0
2   Android           100     100.0

App Version Distribution:


,app_version,unique_users,user_pct
0,2.0.5,61,61.0
1,1.6.6,61,61.0
2,2.6.9,61,61.0
3,2.0.8,58,58.0
4,1.2.8,58,58.0
...,...,...,...
195,2.4.5,38,38.0
196,2.5.0,38,38.0
197,2.5.7,38,38.0
198,2.1.1,37,37.0


In [58]:
# Define drop-off metrics by device_os and app_version
drop_off_metrics = conn.execute("""
    WITH user_first_sessions AS (
        SELECT 
            user_id,
            MIN(session_id) as first_session_id,
            SUBSTRING(MIN(timestamp), 1, 10) as first_session_date
        FROM trusted_events
        GROUP BY user_id
    ),
    user_device_info AS (
        -- Get device info from first session
        SELECT DISTINCT
            ufs.user_id,
            e.device_os,
            e.app_version
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND ufs.first_session_id = e.session_id
    ),
    first_session_watch_times AS (
        SELECT
            ufs.user_id,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as first_session_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e
            ON ufs.user_id = e.user_id
            AND ufs.first_session_id = e.session_id
        GROUP BY ufs.user_id
    ),
    user_session_counts AS (
        SELECT 
            user_id,
            COUNT(DISTINCT session_id) as total_sessions
        FROM trusted_events
        GROUP BY user_id
    ),
    day1_retention AS (
        -- Check if user returned within 24 hours
        SELECT 
            ufs.user_id,
            CASE WHEN COUNT(DISTINCT e.session_id) > 0 THEN 1 ELSE 0 END as returned_day1
        FROM user_first_sessions ufs
        LEFT JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND e.session_id > ufs.first_session_id
            AND SUBSTRING(e.timestamp, 1, 10) = CAST(DATE_ADD(CAST(ufs.first_session_date AS DATE), INTERVAL 1 DAY) AS VARCHAR)
        GROUP BY ufs.user_id
    )
    SELECT 
        udi.device_os,
        udi.app_version,
        COUNT(DISTINCT udi.user_id) as total_users,
        
        -- Drop-off metric 1: No second session
        COUNT(DISTINCT CASE WHEN usc.total_sessions = 1 THEN udi.user_id END) as users_single_session,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN usc.total_sessions = 1 THEN udi.user_id END) / COUNT(DISTINCT udi.user_id), 1) as single_session_rate_pct,
        
        -- Drop-off metric 2: Low first session watch time (<5 seconds)
        COUNT(DISTINCT CASE WHEN fswt.first_session_watch_time < 5 THEN udi.user_id END) as users_low_watch_time,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN fswt.first_session_watch_time < 5 THEN udi.user_id END) / COUNT(DISTINCT udi.user_id), 1) as low_watch_time_rate_pct,
        
        -- Drop-off metric 3: No day 1 retention
        COUNT(DISTINCT CASE WHEN dr.returned_day1 = 0 THEN udi.user_id END) as users_no_day1_return,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN dr.returned_day1 = 0 THEN udi.user_id END) / COUNT(DISTINCT udi.user_id), 1) as no_day1_return_rate_pct,
        
        -- Average metrics for comparison
        ROUND(AVG(fswt.first_session_watch_time), 1) as avg_first_session_watch_time,
        ROUND(AVG(usc.total_sessions), 1) as avg_total_sessions
        
    FROM user_device_info udi
    LEFT JOIN first_session_watch_times fswt ON udi.user_id = fswt.user_id
    LEFT JOIN user_session_counts usc ON udi.user_id = usc.user_id
    LEFT JOIN day1_retention dr ON udi.user_id = dr.user_id
    GROUP BY udi.device_os, udi.app_version
    HAVING COUNT(DISTINCT udi.user_id) >= 5  -- Only include combinations with 5+ users
    ORDER BY single_session_rate_pct DESC
""").df()

print("Drop-off Metrics by Device OS & App Version:")
drop_off_metrics

Drop-off Metrics by Device OS & App Version:


,device_os,app_version,total_users,users_single_session,single_session_rate_pct,users_low_watch_time,low_watch_time_rate_pct,users_no_day1_return,no_day1_return_rate_pct,avg_first_session_watch_time,avg_total_sessions
0,iOS,2.0.1,5,0,0.0,3,60.0,0,0.0,9.2,12.8
1,Windows,2.7.2,6,0,0.0,0,0.0,0,0.0,18.3,10.7
2,Windows,2.8.6,5,0,0.0,2,40.0,0,0.0,18.4,10.0
3,iOS,2.4.9,6,0,0.0,1,16.7,0,0.0,11.8,9.5
4,Windows,1.1.1,6,0,0.0,0,0.0,0,0.0,16.3,10.8
5,Android,1.6.6,8,0,0.0,1,12.5,0,0.0,13.5,10.5
6,iOS,2.8.1,5,0,0.0,0,0.0,0,0.0,15.4,9.6
7,Windows,1.6.7,5,0,0.0,0,0.0,0,0.0,13.2,9.2
8,iOS,2.6.9,5,0,0.0,1,20.0,0,0.0,9.8,10.2
9,Windows,1.3.7,5,0,0.0,0,0.0,0,0.0,9.4,9.8


In [59]:
# Calculate overall drop-off benchmarks for comparison
overall_benchmarks = conn.execute("""
    WITH user_first_sessions AS (
        SELECT 
            user_id,
            MIN(session_id) as first_session_id,
            SUBSTRING(MIN(timestamp), 1, 10) as first_session_date
        FROM trusted_events
        GROUP BY user_id
    ),
    first_session_watch_times AS (
        SELECT
            ufs.user_id,
            SUM(CASE WHEN e.event_name = 'watch_time' THEN CAST(e.value AS DOUBLE) ELSE 0 END) as first_session_watch_time
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e
            ON ufs.user_id = e.user_id
            AND ufs.first_session_id = e.session_id
        GROUP BY ufs.user_id
    ),
    user_session_counts AS (
        SELECT 
            user_id,
            COUNT(DISTINCT session_id) as total_sessions
        FROM trusted_events
        GROUP BY user_id
    ),
    day1_retention AS (
        SELECT 
            ufs.user_id,
            CASE WHEN COUNT(DISTINCT e.session_id) > 0 THEN 1 ELSE 0 END as returned_day1
        FROM user_first_sessions ufs
        LEFT JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND e.session_id > ufs.first_session_id
            AND SUBSTRING(e.timestamp, 1, 10) = CAST(DATE_ADD(CAST(ufs.first_session_date AS DATE), INTERVAL 1 DAY) AS VARCHAR)
        GROUP BY ufs.user_id
    )
    SELECT 
        'OVERALL' as category,
        COUNT(DISTINCT ufs.user_id) as total_users,
        COUNT(DISTINCT CASE WHEN usc.total_sessions = 1 THEN ufs.user_id END) as users_single_session,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN usc.total_sessions = 1 THEN ufs.user_id END) / COUNT(DISTINCT ufs.user_id), 1) as single_session_rate_pct,
        COUNT(DISTINCT CASE WHEN fswt.first_session_watch_time < 5 THEN ufs.user_id END) as users_low_watch_time,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN fswt.first_session_watch_time < 5 THEN ufs.user_id END) / COUNT(DISTINCT ufs.user_id), 1) as low_watch_time_rate_pct,
        COUNT(DISTINCT CASE WHEN dr.returned_day1 = 0 THEN ufs.user_id END) as users_no_day1_return,
        ROUND(100.0 * COUNT(DISTINCT CASE WHEN dr.returned_day1 = 0 THEN ufs.user_id END) / COUNT(DISTINCT ufs.user_id), 1) as no_day1_return_rate_pct,
        ROUND(AVG(fswt.first_session_watch_time), 1) as avg_first_session_watch_time,
        ROUND(AVG(usc.total_sessions), 1) as avg_total_sessions
    FROM user_first_sessions ufs
    LEFT JOIN first_session_watch_times fswt ON ufs.user_id = fswt.user_id
    LEFT JOIN user_session_counts usc ON ufs.user_id = usc.user_id
    LEFT JOIN day1_retention dr ON ufs.user_id = dr.user_id
""").df()

print("Overall Drop-off Benchmarks:")
overall_benchmarks

Overall Drop-off Benchmarks:


,category,total_users,users_single_session,single_session_rate_pct,users_low_watch_time,low_watch_time_rate_pct,users_no_day1_return,no_day1_return_rate_pct,avg_first_session_watch_time,avg_total_sessions
0,OVERALL,100,0,0.0,19,19.0,0,0.0,11.6,10.1


In [60]:
# Identify abnormally high drop-off combinations
# Calculate z-scores and identify outliers
import numpy as np

# Get overall benchmarks
overall_single_session_rate = overall_benchmarks['single_session_rate_pct'].iloc[0]
overall_low_watch_rate = overall_benchmarks['low_watch_time_rate_pct'].iloc[0]
overall_no_day1_rate = overall_benchmarks['no_day1_return_rate_pct'].iloc[0]

print(f"Overall Benchmarks:")
print(f"  Single session rate: {overall_single_session_rate}%")
print(f"  Low watch time rate: {overall_low_watch_rate}%")
print(f"  No day 1 return rate: {overall_no_day1_rate}%")
print()

# Calculate deviations from overall averages
drop_off_metrics_with_scores = drop_off_metrics.copy()
drop_off_metrics_with_scores['single_session_deviation'] = drop_off_metrics_with_scores['single_session_rate_pct'] - overall_single_session_rate
drop_off_metrics_with_scores['low_watch_deviation'] = drop_off_metrics_with_scores['low_watch_time_rate_pct'] - overall_low_watch_rate
drop_off_metrics_with_scores['no_day1_deviation'] = drop_off_metrics_with_scores['no_day1_return_rate_pct'] - overall_no_day1_rate

# Calculate composite drop-off score (higher = worse)
drop_off_metrics_with_scores['composite_drop_off_score'] = (
    drop_off_metrics_with_scores['single_session_deviation'] * 0.4 +  # Weight single session highly
    drop_off_metrics_with_scores['low_watch_deviation'] * 0.3 +       # Weight low engagement
    drop_off_metrics_with_scores['no_day1_deviation'] * 0.3           # Weight day 1 retention
)

# Sort by worst drop-off
drop_off_analysis = drop_off_metrics_with_scores.sort_values('composite_drop_off_score', ascending=False)

print("Drop-off Analysis (sorted by worst performers):")
print("Composite Score = 0.4×single_session_dev + 0.3×low_watch_dev + 0.3×no_day1_dev")
print("Higher score = worse drop-off performance")
print()
drop_off_analysis[['device_os', 'app_version', 'total_users', 'single_session_rate_pct', 
                  'low_watch_time_rate_pct', 'no_day1_return_rate_pct', 'composite_drop_off_score']].head(10)

Overall Benchmarks:
  Single session rate: 0.0%
  Low watch time rate: 19.0%
  No day 1 return rate: 0.0%

Drop-off Analysis (sorted by worst performers):
Composite Score = 0.4×single_session_dev + 0.3×low_watch_dev + 0.3×no_day1_dev
Higher score = worse drop-off performance



,device_os,app_version,total_users,single_session_rate_pct,low_watch_time_rate_pct,no_day1_return_rate_pct,composite_drop_off_score
0,iOS,2.0.1,5,0.0,60.0,0.0,12.3
19,iOS,1.4.5,6,0.0,50.0,0.0,9.3
45,iOS,2.7.2,5,0.0,40.0,0.0,6.3
36,iOS,1.5.9,5,0.0,40.0,0.0,6.3
35,Windows,1.6.1,5,0.0,40.0,0.0,6.3
31,Windows,2.2.8,5,0.0,40.0,0.0,6.3
29,Android,1.5.2,5,0.0,40.0,0.0,6.3
26,Windows,1.4.1,5,0.0,40.0,0.0,6.3
20,Android,2.1.5,5,0.0,40.0,0.0,6.3
50,Android,2.0.2,5,0.0,40.0,0.0,6.3


In [61]:
# Detailed analysis of the worst performing combination
worst_combo = drop_off_analysis.iloc[0]
print("WORST PERFORMING COMBINATION:")
print(f"Device OS: {worst_combo['device_os']}")
print(f"App Version: {worst_combo['app_version']}")
print(f"Total Users: {worst_combo['total_users']}")
print()
print("Drop-off Metrics vs Overall Average:")
print(f"  Single Session Rate: {worst_combo['single_session_rate_pct']}% (vs {overall_single_session_rate}% overall) - {worst_combo['single_session_deviation']:+.1f}pp")
print(f"  Low Watch Time Rate: {worst_combo['low_watch_time_rate_pct']}% (vs {overall_low_watch_rate}% overall) - {worst_combo['low_watch_deviation']:+.1f}pp")
print(f"  No Day 1 Return Rate: {worst_combo['no_day1_return_rate_pct']}% (vs {overall_no_day1_rate}% overall) - {worst_combo['no_day1_deviation']:+.1f}pp")
print(f"  Composite Drop-off Score: {worst_combo['composite_drop_off_score']:.1f}")
print()
print("Engagement Quality:")
print(f"  Avg First Session Watch Time: {worst_combo['avg_first_session_watch_time']} seconds (vs {overall_benchmarks['avg_first_session_watch_time'].iloc[0]} overall)")
print(f"  Avg Total Sessions: {worst_combo['avg_total_sessions']} (vs {overall_benchmarks['avg_total_sessions'].iloc[0]} overall)")

# Show user IDs for investigation
worst_combo_users = conn.execute(f"""
    WITH user_first_sessions AS (
        SELECT 
            user_id,
            MIN(session_id) as first_session_id
        FROM trusted_events
        GROUP BY user_id
    ),
    user_device_info AS (
        SELECT DISTINCT
            ufs.user_id,
            e.device_os,
            e.app_version
        FROM user_first_sessions ufs
        INNER JOIN trusted_events e 
            ON ufs.user_id = e.user_id 
            AND ufs.first_session_id = e.session_id
    )
    SELECT user_id
    FROM user_device_info
    WHERE device_os = '{worst_combo['device_os']}' 
    AND app_version = '{worst_combo['app_version']}'
    ORDER BY user_id
    LIMIT 10
""").df()

print()
print("Sample Users for Investigation:")
print(worst_combo_users['user_id'].tolist())

WORST PERFORMING COMBINATION:
Device OS: iOS
App Version: 2.0.1
Total Users: 5

Drop-off Metrics vs Overall Average:
  Single Session Rate: 0.0% (vs 0.0% overall) - +0.0pp
  Low Watch Time Rate: 60.0% (vs 19.0% overall) - +41.0pp
  No Day 1 Return Rate: 0.0% (vs 0.0% overall) - +0.0pp
  Composite Drop-off Score: 12.3

Engagement Quality:
  Avg First Session Watch Time: 9.2 seconds (vs 11.6 overall)
  Avg Total Sessions: 12.8 (vs 10.1 overall)

Sample Users for Investigation:
['user_25', 'user_46', 'user_48', 'user_67', 'user_95']


In [63]:
print("Q3 CONCLUSION:")
print("=================")
print(f"YES - There is abnormally high drop-off in: {worst_combo['device_os']} + {worst_combo['app_version']}")
print()
print("Key Findings:")
print(f"- This combination affects {worst_combo['total_users']} users")
print(f"- Has a composite drop-off score of {worst_combo['composite_drop_off_score']:.1f} (higher = worse)")
print()
print("Specific Drop-off Issues:")
print(f"- Single Session Rate: {worst_combo['single_session_rate_pct']}% (vs {overall_single_session_rate}% overall)")
print(f"  - {worst_combo['single_session_deviation']:+.1f} percentage points worse than average")
print(f"- Low Watch Time Rate: {worst_combo['low_watch_time_rate_pct']}% (vs {overall_low_watch_rate}% overall)")
print(f"  - {worst_combo['low_watch_deviation']:+.1f} percentage points worse than average")
print(f"- No Day 1 Return: {worst_combo['no_day1_return_rate_pct']}% (vs {overall_no_day1_rate}% overall)")
print(f"  - {worst_combo['no_day1_deviation']:+.1f} percentage points worse than average")
print()
print("Impact on Engagement:")
print(f"- Average first session watch time: {worst_combo['avg_first_session_watch_time']} seconds")
print(f"  (vs {overall_benchmarks['avg_first_session_watch_time'].iloc[0]} seconds overall)")
print(f"- Average total sessions: {worst_combo['avg_total_sessions']}")
print(f"  (vs {overall_benchmarks['avg_total_sessions'].iloc[0]} sessions overall)")
print()
print("Recommendations:")
print(f"1. Investigate technical issues with {worst_combo['device_os']} + {worst_combo['app_version']}")
print("2. Check for bugs, crashes, or performance issues on this combination")
print("3. Consider targeted fixes or app version updates")
print("4. Monitor user feedback from affected users")
print()
print(f"*** FINAL Q3 ANSWER: {worst_combo['device_os']} + {worst_combo['app_version']} ***")

Q3 CONCLUSION:
YES - There is abnormally high drop-off in: iOS + 2.0.1

Key Findings:
- This combination affects 5 users
- Has a composite drop-off score of 12.3 (higher = worse)

Specific Drop-off Issues:
- Single Session Rate: 0.0% (vs 0.0% overall)
  - +0.0 percentage points worse than average
- Low Watch Time Rate: 60.0% (vs 19.0% overall)
  - +41.0 percentage points worse than average
- No Day 1 Return: 0.0% (vs 0.0% overall)
  - +0.0 percentage points worse than average

Impact on Engagement:
- Average first session watch time: 9.2 seconds
  (vs 11.6 seconds overall)
- Average total sessions: 12.8
  (vs 10.1 sessions overall)

Recommendations:
1. Investigate technical issues with iOS + 2.0.1
2. Check for bugs, crashes, or performance issues on this combination
3. Consider targeted fixes or app version updates
4. Monitor user feedback from affected users

*** FINAL Q3 ANSWER: iOS + 2.0.1 ***
